In [1]:
# from pyspark.sql import SparkSession

In [1]:
%load_ext sparksql_magic

#### Following code is used to initilize the Spark Session. Delta lake package is used while creating the spark session, which will help to save spark dataframe as Delta Table. Unity Catalog is used as default catalog instead of Delta Catalog and Minio as Object Storage
- This code is commented since Jupyter is started with Pyspark session using the same configutation mentioned here. The Pyspark Jupyter script can be found in docker entryfile of unity catalog present in folder **"install-and-config\2-spark-delta-unity-minio-integration"**..
  

In [1]:
# # Initialize SparkSession
# spark = SparkSession.builder \
#     .appName("jupyter-unity-catalog-minio-session") \
#     .master("spark://spark-master:7077") \
#     .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.1,io.unitycatalog:unitycatalog-spark_2.12:0.2.0,org.apache.hadoop:hadoop-aws:3.3.4") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "io.unitycatalog.spark.UCSingleCatalog") \
#     .config("spark.sql.catalog.unity", "io.unitycatalog.spark.UCSingleCatalog") \
#     .config("spark.sql.catalog.unity.uri", "http://unity-catalog:8080") \
#     .config("spark.sql.catalog.unity.token", "") \
#     .config("spark.sql.defaultCatalog", "unity") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "root") \
#     .config("spark.hadoop.fs.s3a.secret.key", "jerinminioserver") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .getOrCreate()


In [9]:
# To stop SparkSession
# spark.stop()

In [85]:
# Get spark session config details
# spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.jars',
  'file:///root/.ivy2/jars/io.delta_delta-spark_2.12-3.2.0.jar,file:///root/.ivy2/jars/io.unitycatalog_unitycatalog-spark_2.12-0.2.0.jar,file:///root/.ivy2/jars/io.delta_delta-storage-3.2.0.jar,file:///root/.ivy2/jars/org.antlr_antlr4-runtime-4.9.3.jar,file:///root/.ivy2/jars/io.unitycatalog_unitycatalog-client-0.2.0.jar,file:///root/.ivy2/jars/org.slf4j_slf4j-api-2.0.13.jar,file:///root/.ivy2/jars/org.apache.logging.log4j_log4j-slf4j2-impl-2.23.1.jar,file:///root/.ivy2/jars/org.apache.logging.log4j_log4j-api-2.23.1.jar,file:///root/.ivy2/jars/com.fasterxml.jackson.core_jackson-databind-2.15.0.jar,file:///root/.ivy2/jars/com.fasterxml.jackson.module_jackson-module-scala_2.12-2.15.0.jar,file:///root/.ivy2/jars/com.fasterxml.jackson.core_jackson-annotations-2.15.0.jar,file:///root/.ivy2/jars/com.fasterxml.jackson.core_jackson-core-2.15.0.jar,file:///root/.ivy2/jars/com.fasterxml.jackson.dataformat_jackson-dataformat-xml-2.15.0.jar,file

In [1]:
# Example DataFrame creation
data = [("Jerin", 29), ("Aayush", 35), ("Neeraj", 28)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

In [2]:
df.show()

+------+---+
|  Name|Age|
+------+---+
| Jerin| 29|
|Aayush| 35|
|Neeraj| 28|
+------+---+



In [3]:
# unmanaged delta table - Save to minio
df.write.format("delta").save("s3a://table-bucket/external_tables/testUnmanagedTableOnMinio")

25/01/09 07:42:34 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [4]:
# Read delta table from minio
spark.sql("""select * from delta.`s3a://table-bucket/external_tables/testUnmanagedTableOnMinio`;""").show()

+------+---+
|  Name|Age|
+------+---+
|Aayush| 35|
|Neeraj| 28|
| Jerin| 29|
+------+---+



In [9]:
%%sparksql
SHOW CATALOGS;

catalog
spark_catalog
unity


In [10]:
%%sparksql
SHOW SCHEMAS IN unity;

namespace
default


In [5]:
%%sparksql
SHOW TABLES IN unity.default;

namespace,tableName,isTemporary
default,marksheet,False
default,marksheet_uniform,False
default,numbers,False
default,testUnityCatalogOnMinio,False
default,testUnityCatalogOnMinio123,False
default,user_countries,False


In [6]:
%%sparksql
USE unity.default

In [13]:
%%sparksql

SELECT current_schema()

current_database()
default


In [7]:
# create bucket ("table-bucket") in Minio, if not exists
spark.sql("""
CREATE TABLE unity.default.testUnityCatalogOnMinio1234 (
    id INT, 
    desc STRING
) 
USING delta 
LOCATION 's3a://table-bucket/minio_data/tables/testUnityCatalogOnMinio1234';""")

DataFrame[]

In [11]:
%%sparksql

CREATE TABLE unity.default.testUnityCatalogOnMinio123456 (
    id INT, 
    desc STRING
) 
USING delta 
LOCATION 's3a://table-bucket/minio_data/tables/testUnityCatalogOnMinio123456';

In [9]:
%%sparksql

Insert into unity.default.testUnityCatalogOnMinio12345 Values (1,'Jerin'),(2,'Neeraj'), (3,'Atul'),(4,'Priti'),(5,'Rahul'),(6,'Chinmay'),(7,'Ashu');

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`testUnityCatalogOnMinio12345` doesn't exist.;
AppendData RelationV2[] unity.default.testUnityCatalogOnMinio12345 unity.default.testUnityCatalogOnMinio12345, false
+- LocalRelation [col1#730, col2#731]


In [12]:
%%sparksql

Select * from unity.default.testTableFromBash;

Py4JJavaError: An error occurred while calling o44.sql.
: io.unitycatalog.client.ApiException: generateTemporaryTableCredentials call failed with: 400 - {"error_code":"FAILED_PRECONDITION","details":[{"reason":"FAILED_PRECONDITION","metadata":{},"@type":"google.rpc.ErrorInfo"}],"stack_trace":null,"message":"S3 bucket configuration not found."}
	at io.unitycatalog.client.api.TemporaryCredentialsApi.getApiException(TemporaryCredentialsApi.java:78)
	at io.unitycatalog.client.api.TemporaryCredentialsApi.generateTemporaryTableCredentialsWithHttpInfo(TemporaryCredentialsApi.java:266)
	at io.unitycatalog.client.api.TemporaryCredentialsApi.generateTemporaryTableCredentials(TemporaryCredentialsApi.java:244)
	at io.unitycatalog.spark.UCProxy.loadTable(UCSingleCatalog.scala:266)
	at org.apache.spark.sql.connector.catalog.DelegatingCatalogExtension.loadTable(DelegatingCatalogExtension.java:73)
	at org.apache.spark.sql.delta.catalog.DeltaCatalog.super$loadTable(DeltaCatalog.scala:222)
	at org.apach